In [2]:
%%pyspark
df = spark.read.load('abfss://files@datalake0wib864.dfs.core.windows.net/data/2019.csv', format='csv', header=True
## If header exists uncomment line below
##, header=True
)
display(df.limit(10))

StatementMeta(spark0wib864, 0, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 39b1f340-1dc6-469d-9d4a-bd377749c4e0)

In [5]:
from pyspark.sql.functions import split, col
transformed_df=df.withColumn("FirstName",split(col("CustomerName")," ").getItem(0)).withColumn("LastName",split(col("CustomerName")," ").getItem(1))

StatementMeta(spark0wib864, 0, 6, Finished, Available)

In [6]:
display(transformed_df)

StatementMeta(spark0wib864, 0, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, cdf38c69-fa48-4aa1-99a3-be8a46c379d3)

In [7]:
transformed_df.drop("CustomerName")

StatementMeta(spark0wib864, 0, 8, Finished, Available)

DataFrame[SalesOrderNumber: string, SalesOrderLineNumber: string, OrderDate: string, EmailAddress: string, Item: string, Quantity: string, UnitPrice: string, TaxAmount: string, FirstName: string, LastName: string]

In [9]:
transformed_df.write.mode("overwrite").parquet("/transformed_data/orders.parquet")

StatementMeta(spark0wib864, 0, 10, Finished, Available)

In [18]:
from pyspark.sql.functions import year, col

dated_df = df.withColumn("Year", year(col("OrderDate")))

StatementMeta(spark0wib864, 0, 19, Finished, Available)

In [25]:
dated_df.write.partitionBy("Year").mode("overwrite").parquet("/data/datos.parquet")


StatementMeta(spark0wib864, 0, 26, Finished, Available)

In [28]:
order_2019 = spark.read.parquet("/data/datos.parquet/Year=2019")


StatementMeta(spark0wib864, 0, 29, Finished, Available)

In [29]:
display(order_2019)

StatementMeta(spark0wib864, 0, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, a29c70b2-d30c-468e-b9c4-8b0300f7be61)

In [30]:
df.write.saveAsTable('sales_orders', format='parquet', mode='overwrite', path='/sales_orders_table')


StatementMeta(spark0wib864, 0, 31, Finished, Available)

In [31]:

sql_transform = spark.sql("SELECT *, YEAR(OrderDate) AS Year, MONTH(OrderDate) AS Month FROM sales_orders")

sql_transform.write.partitionBy("Year","Month").saveAsTable('transformed_orders', format='parquet', mode='overwrite', path='/transformed_orders_table')

StatementMeta(spark0wib864, 0, 32, Finished, Available)

In [34]:
%%sql

SELECT * FROM transformed_orders
where Month=7;

StatementMeta(spark0wib864, 0, 35, Finished, Available)

<Spark SQL result set with 289 rows and 11 fields>

In [36]:
df.createOrReplaceTempView("dfsql")

StatementMeta(spark0wib864, 0, 37, Finished, Available)

In [40]:
%%sql

SELECT *, month(OrderDate) as Month FROM dfsql

StatementMeta(spark0wib864, 0, 41, Finished, Available)

<Spark SQL result set with 1000 rows and 10 fields>